<a href="https://colab.research.google.com/github/Virtuoso633/FOOD_DELIVERY_GOOGLE_ADK/blob/main/food_delivery_google_adk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# @title Step 0: Setup and Installation
# Install ADK and LiteLLM for multi-model support

!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

Installation complete.


In [54]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [55]:
# @title Configure API Keys (Replace with your actual keys!)

# --- IMPORTANT: Replace placeholders with your real API keys ---

# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY" # <--- REPLACE(PREFERRED)

# [Optional]
# OpenAI API Key (Get from OpenAI Platform: https://platform.openai.com/api-keys)
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY' # <--- REPLACE

# [Optional]
# Anthropic API Key (Get from Anthropic Console: https://console.anthropic.com/settings/keys)
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_API_KEY' # <--- REPLACE

# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Security Note:** It's best practice to manage API keys securely (e.g., using Colab Secrets or environment variables) rather than hardcoding them directly in the notebook. Replace the placeholder strings above.

API Keys Set:
Google API Key set: Yes
OpenAI API Key set: No (REPLACE PLACEHOLDER!)
Anthropic API Key set: No (REPLACE PLACEHOLDER!)


In [56]:
from typing import Optional
from google.adk.tools.tool_context import ToolContext

def get_user_location(user_input: Optional[str] = None, tool_context: ToolContext = None) -> dict:
    """Determines the user's location for food delivery.

    Args:
        user_input (Optional[str]): User-provided location or address
        tool_context (ToolContext): Session context (auto-injected by ADK)

    Returns:
        dict: Location information including coordinates and formatted address
    """
    print(f"--- Tool: get_user_location called with input: {user_input} ---")

    # Mock location data for demonstration
    if user_input:
        mock_locations = {
            "downtown": {"lat": 40.7589, "lng": -73.9851, "address": "123 Downtown Ave, New York, NY"},
            "uptown": {"lat": 40.7831, "lng": -73.9712, "address": "456 Uptown St, New York, NY"},
            "brooklyn": {"lat": 40.6782, "lng": -73.9442, "address": "789 Brooklyn Blvd, Brooklyn, NY"}
        }
        normalized_input = user_input.lower().strip()
        location_data = mock_locations.get(normalized_input,
            {"lat": 40.7488, "lng": -73.9857, "address": user_input})
    else:
        location_data = {"lat": 40.7488, "lng": -73.9857, "address": "Current Location, New York, NY"}

    # Store in context if available
    if tool_context:
        tool_context.state["current_location"] = location_data

    return {
        "status": "success",
        "location": location_data,
        "delivery_available": True
    }

def get_restaurants(user_location: dict, tool_context: ToolContext) -> dict:
    """Finds restaurants near the user's location with ratings and available coupons.

    Args:
        user_location (dict): Location data from get_user_location tool
        tool_context (ToolContext): Access to session state for preferences

    Returns:
        dict: Restaurant listings with ratings, cuisine types, and available coupons
    """
    print(f"--- Tool: get_restaurants called for location: {user_location.get('address', 'Unknown')} ---")

    # Read user preferences from state
    preferred_cuisine = tool_context.state.get("preferred_cuisine", "any")
    dietary_restrictions = tool_context.state.get("dietary_restrictions", [])

    mock_restaurants = [
        {"id": "r1", "name": "Mario's Pizza", "cuisine": "Italian", "rating": 4.5,
         "delivery_time": "30-45 min", "coupons": ["20% off orders over $25"]},
        {"id": "r2", "name": "Sakura Sushi", "cuisine": "Japanese", "rating": 4.7,
         "delivery_time": "25-40 min", "coupons": ["Free delivery"]},
        {"id": "r3", "name": "Burger Palace", "cuisine": "American", "rating": 4.2,
         "delivery_time": "35-50 min", "coupons": ["Buy 1 Get 1 Free"]}
    ]

    # Store restaurant data in session state
    tool_context.state["available_restaurants"] = mock_restaurants
    tool_context.state["current_location"] = user_location

    return {
        "status": "success",
        "restaurants": mock_restaurants,
        "location": user_location,
        "coupons_available": True
    }

def select_restaurant(restaurant_name: str, tool_context: ToolContext) -> dict:
    """Selects a restaurant based on user choice.

    Args:
        restaurant_name (str): Name of the restaurant to select
        tool_context (ToolContext): Session state access

    Returns:
        dict: Selected restaurant information
    """
    print(f"--- Tool: select_restaurant called with restaurant: {restaurant_name} ---")

    restaurants = tool_context.state.get("available_restaurants", [])

    # Find restaurant by name (case-insensitive)
    selected_restaurant = None
    for restaurant in restaurants:
        if restaurant_name.lower() in restaurant["name"].lower():
            selected_restaurant = restaurant
            break

    if not selected_restaurant and restaurants:
        selected_restaurant = restaurants[0]  # Default to first if not found

    if selected_restaurant:
        tool_context.state["selected_restaurant"] = selected_restaurant
        return {
            "status": "success",
            "restaurant": selected_restaurant
        }
    else:
        return {
            "status": "error",
            "message": f"Restaurant '{restaurant_name}' not found"
        }

def get_menu(restaurant_name: str, tool_context: ToolContext) -> dict:
    """Retrieves the menu for the selected restaurant.

    Args:
        restaurant_name (str): Name of the restaurant
        tool_context (ToolContext): Session state access

    Returns:
        dict: Complete menu with sections and items
    """
    print(f"--- Tool: get_menu called for {restaurant_name} ---")

    mock_menus = {
        "mario's pizza": {
            "sections": {
                "appetizers": [
                    {"id": "a1", "name": "Garlic Bread", "price": 8.99, "description": "Fresh baked with herbs"},
                    {"id": "a2", "name": "Caesar Salad", "price": 12.99, "description": "Crisp romaine, parmesan"}
                ],
                "mains": [
                    {"id": "m1", "name": "Margherita Pizza", "price": 18.99, "description": "Fresh mozzarella, basil"},
                    {"id": "m2", "name": "Pepperoni Pizza", "price": 21.99, "description": "Classic pepperoni"}
                ],
                "desserts": [
                    {"id": "d1", "name": "Tiramisu", "price": 7.99, "description": "Classic Italian dessert"}
                ]
            }
        }
    }

    normalized_name = restaurant_name.lower()
    menu = mock_menus.get(normalized_name, mock_menus["mario's pizza"])  # Default menu

    tool_context.state["current_menu"] = menu

    return {"status": "success", "menu": menu, "restaurant": restaurant_name}

def select_menu_section(section_name: str, tool_context: ToolContext) -> dict:
    """Helps user navigate menu sections based on preferences.

    Args:
        section_name (str): Menu section name (e.g., "mains", "appetizers", "desserts")
        tool_context (ToolContext): Session state access

    Returns:
        dict: Selected menu section with available dishes
    """
    print(f"--- Tool: select_menu_section called with section: {section_name} ---")

    menu = tool_context.state.get("current_menu", {})
    sections = menu.get("sections", {})

    # Normalize section name
    section_mapping = {
        "appetizer": "appetizers",
        "starter": "appetizers",
        "main": "mains",
        "mains": "mains",
        "main dishes": "mains",
        "entree": "mains",
        "dessert": "desserts",
        "sweet": "desserts"
    }

    normalized_section = section_name.lower()
    actual_section = section_mapping.get(normalized_section, normalized_section)

    dishes = sections.get(actual_section, [])

    tool_context.state["selected_section"] = actual_section
    tool_context.state["available_dishes"] = dishes

    return {
        "status": "success",
        "section": actual_section,
        "dishes": dishes
    }

def select_dishes(dish_name: str, quantity: int, special_requests: Optional[str] = None, tool_context: ToolContext = None) -> dict:
    """Finalizes dish selection with quantity and special requests.

    Args:
        dish_name (str): Name of the dish to order
        quantity (int): Number of items to order
        special_requests (Optional[str]): Any special preparation requests
        tool_context (ToolContext): Session state access

    Returns:
        dict: Final dish selection with pricing
    """
    print(f"--- Tool: select_dishes called for {dish_name}, quantity: {quantity} ---")

    if not tool_context:
        return {"status": "error", "message": "Tool context required"}

    dishes = tool_context.state.get("available_dishes", [])

    # Find dish by name
    selected_dish = None
    for dish in dishes:
        if dish_name.lower() in dish["name"].lower():
            selected_dish = dish
            break

    if not selected_dish and dishes:
        selected_dish = dishes[0]  # Default to first dish

    if not selected_dish:
        return {"status": "error", "message": "No dishes available"}

    total_price = selected_dish.get("price", 0) * quantity

    order_item = {
        "dish": selected_dish,
        "quantity": quantity,
        "special_requests": special_requests or "",
        "item_total": total_price
    }

    # Update order state
    current_order = tool_context.state.get("current_order", [])
    current_order.append(order_item)
    tool_context.state["current_order"] = current_order
    tool_context.state["order_total"] = sum(item["item_total"] for item in current_order)

    return {"status": "success", "dish": order_item, "order_total": tool_context.state["order_total"]}

def pay(payment_method: str, billing_address: str, tool_context: ToolContext) -> dict:
    """Processes payment for the food order.

    Args:
        payment_method (str): Payment method (e.g., "credit_card", "debit_card", "paypal")
        billing_address (str): Billing address for payment
        tool_context (ToolContext): Session state access

    Returns:
        dict: Payment confirmation and order tracking information
    """
    print(f"--- Tool: pay called with payment method: {payment_method} ---")

    current_order = tool_context.state.get("current_order", [])
    order_total = tool_context.state.get("order_total", 0)

    if not current_order:
        return {"status": "error", "message": "No items in order"}

    # Simulate payment processing
    order_id = f"FD{hash(str(current_order)) % 100000:05d}"

    payment_confirmation = {
        "order_id": order_id,
        "status": "confirmed",
        "total_amount": order_total,
        "estimated_delivery": "35-50 minutes",
        "payment_method": payment_method,
        "billing_address": billing_address
    }

    # Store confirmation in state
    tool_context.state["payment_confirmation"] = payment_confirmation
    tool_context.state["order_status"] = "confirmed"

    return {"status": "success", "confirmation": payment_confirmation}


In [58]:

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# Update all agents to use the higher rate limit model
location_agent = Agent(
    name="location_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Determines user location for food delivery services",
    instruction="You are the Location Agent. Your ONLY task is to determine the user's location "
                "using the 'get_user_location' tool. Handle location-related queries only.",
    tools=[get_user_location]
)

restaurant_discovery_agent = Agent(
    name="restaurant_discovery_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Discovers restaurants near user location",
    instruction="You are the Restaurant Discovery Agent. Use 'get_restaurants' to find restaurants "
                "near the user's location. Only handle restaurant search queries.",
    tools=[get_restaurants]
)

restaurant_selection_agent = Agent(
    name="restaurant_selection_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Handles restaurant selection from available options",
    instruction="You are the Restaurant Selection Agent. Use 'select_restaurant' to help users "
                "choose a specific restaurant. Only handle restaurant selection tasks.",
    tools=[select_restaurant]
)

menu_retrieval_agent = Agent(
    name="menu_retrieval_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Retrieves restaurant menus",
    instruction="You are the Menu Retrieval Agent. Use 'get_menu' to fetch restaurant menus. "
                "Only handle menu display requests.",
    tools=[get_menu]
)

menu_navigation_agent = Agent(
    name="menu_navigation_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Helps users navigate restaurant menu sections",
    instruction="You are the Menu Navigation Agent. Use 'select_menu_section' to help users "
                "choose menu categories. Only handle menu section navigation.",
    tools=[select_menu_section]
)

dish_selection_agent = Agent(
    name="dish_selection_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Handles final dish selection with quantities and special requests",
    instruction="You are the Dish Selection Agent. Use 'select_dishes' to finalize food choices "
                "with quantities and special requests. Only handle dish ordering.",
    tools=[select_dishes]
)

payment_agent = Agent(
    name="payment_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Processes payments securely for food delivery orders",
    instruction="You are the Payment Agent. Use 'pay' to process payments securely. "
                "Only handle payment processing requests.",
    tools=[pay]
)


In [59]:
manager_agent = Agent(
    name="food_delivery_manager",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Main coordinator for food delivery operations",
    instruction="You are the Food Delivery Manager. You MUST ALWAYS delegate tasks to specialist agents. "
                "NEVER use tools directly yourself. Your job is ONLY to route requests: "
                "\n"
                "- Location requests (like 'downtown', 'find my location') → delegate to 'location_agent' "
                "- Restaurant discovery (like 'find restaurants', 'Italian restaurants') → delegate to 'restaurant_discovery_agent' "
                "- Restaurant selection (like 'I want Mario's Pizza', 'order from') → delegate to 'restaurant_selection_agent' "
                "- Menu requests (like 'show menu', 'menu for Mario's') → delegate to 'menu_retrieval_agent' "
                "- Menu navigation (like 'main dishes', 'appetizers section') → delegate to 'menu_navigation_agent' "
                "- Dish ordering (like 'add 2 pizzas', 'order Margherita') → delegate to 'dish_selection_agent' "
                "- Payment (like 'pay', 'process payment', 'credit card') → delegate to 'payment_agent' "
                "\n"
                "You must analyze each request and delegate to the appropriate agent. Do not handle tasks yourself.",
    tools=[],  # Keep empty - Manager only delegates
    sub_agents=[
        location_agent,
        restaurant_discovery_agent,
        restaurant_selection_agent,
        menu_retrieval_agent,
        menu_navigation_agent,
        dish_selection_agent,
        payment_agent
    ]
)


In [60]:
async def test_complete_food_delivery_workflow_enhanced_rate_limiting():
    """Enhanced testing function with aggressive rate limiting protection."""
    print("=" * 60)
    print("STARTING ENHANCED RATE-LIMITED FOOD DELIVERY APPLICATION TEST")
    print("=" * 60)

    session_service = InMemorySessionService()
    session = await session_service.create_session(
        app_name="food_delivery_app",
        user_id="test_user_enhanced",
        session_id="enhanced_workflow_test",
        state={
            "preferred_cuisine": "Italian",
            "dietary_restrictions": [],
            "payment_method": "credit_card"
        }
    )

    runner = Runner(
        agent=manager_agent,
        app_name="food_delivery_app",
        session_service=session_service
    )

    test_scenarios = [
        {
            "step": 1,
            "description": "Location Detection",
            "query": "I'm hungry and want to order food from downtown",
            "expected_agent": "location_agent"
        },
        {
            "step": 2,
            "description": "Restaurant Discovery",
            "query": "Find Italian restaurants with good ratings near me",
            "expected_agent": "restaurant_discovery_agent"
        },
        {
            "step": 3,
            "description": "Restaurant Selection",
            "query": "I want to order from Mario's Pizza",
            "expected_agent": "restaurant_selection_agent"
        },
        {
            "step": 4,
            "description": "Menu Retrieval",
            "query": "Show me the menu for Mario's Pizza",
            "expected_agent": "menu_retrieval_agent"
        },
        {
            "step": 5,
            "description": "Menu Section Navigation",
            "query": "I want to see the main dishes section",
            "expected_agent": "menu_navigation_agent"
        },
        {
            "step": 6,
            "description": "Dish Selection",
            "query": "Add 2 Margherita pizzas to my order with extra cheese",
            "expected_agent": "dish_selection_agent"
        },
        {
            "step": 7,
            "description": "Payment Processing",
            "query": "Process payment with credit card and billing address 123 Main St",
            "expected_agent": "payment_agent"
        }
    ]

    workflow_results = []

    # Longer initial delay for rate limiting
    print("⏰ Waiting 15 seconds before starting to respect rate limits...")
    await asyncio.sleep(15)

    for i, scenario in enumerate(test_scenarios):
        print(f"\n{'='*20} STEP {scenario['step']}: {scenario['description'].upper()} {'='*20}")
        print(f"Query: {scenario['query']}")

        # Increased delay between requests (10 seconds instead of 5)
        if i > 0:
            delay = 10  # 10 seconds between requests for 6 RPM (well under 30 RPM limit)
            print(f"⏰ Waiting {delay} seconds to respect rate limits...")
            await asyncio.sleep(delay)

        try:
            final_response = ""
            delegated_correctly = False
            retry_count = 0
            max_retries = 2

            while retry_count <= max_retries:
                try:
                    start_time = asyncio.get_event_loop().time()

                    async for event in runner.run_async(
                        user_id="test_user_enhanced",
                        session_id="enhanced_workflow_test",
                        new_message=types.Content(role='user', parts=[types.Part(text=scenario['query'])])
                    ):
                        # Check for delegation
                        if hasattr(event, 'author') and event.author == scenario['expected_agent']:
                            delegated_correctly = True
                            print(f"✅ Delegated to: {event.author}")

                        # Capture final response
                        if event.is_final_response() and event.content and event.content.parts:
                            final_response = event.content.parts[0].text

                    end_time = asyncio.get_event_loop().time()
                    print(f"⏱️ Step completed in {end_time - start_time:.1f} seconds")
                    break  # Success, exit retry loop

                except Exception as e:
                    if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                        retry_count += 1
                        if retry_count <= max_retries:
                            backoff_delay = 30 * retry_count  # Exponential backoff: 30s, 60s
                            print(f"⏰ Rate limit hit (attempt {retry_count}). Waiting {backoff_delay} seconds...")
                            await asyncio.sleep(backoff_delay)
                        else:
                            raise  # Re-raise after max retries
                    else:
                        raise  # Re-raise non-rate-limit errors immediately

            status = "PASS" if delegated_correctly else "FAIL"

            result = {
                "step": scenario['step'],
                "description": scenario['description'],
                "status": status,
                "expected_agent": scenario['expected_agent'],
                "delegated_correctly": delegated_correctly,
                "response_preview": final_response[:100] + "..." if len(final_response) > 100 else final_response
            }

            workflow_results.append(result)
            print(f"Status: {result['status']}")
            print(f"Response: {result['response_preview']}")

        except Exception as e:
            print(f"❌ ERROR in Step {scenario['step']}: {str(e)}")
            workflow_results.append({
                "step": scenario['step'],
                "description": scenario['description'],
                "status": "ERROR",
                "error": str(e)
            })

    # Generate final report
    print(f"\n{'='*60}")
    print("ENHANCED RATE-LIMITED TEST RESULTS SUMMARY")
    print(f"{'='*60}")

    passed_steps = sum(1 for r in workflow_results if r.get('status') == 'PASS')
    total_steps = len(workflow_results)

    print(f"Overall Success Rate: {passed_steps}/{total_steps} ({(passed_steps/total_steps)*100:.1f}%)")

    for result in workflow_results:
        status_icon = "✅" if result['status'] == 'PASS' else "❌" if result['status'] == 'ERROR' else "⏰"
        print(f"{status_icon} Step {result['step']}: {result['description']} - {result['status']}")
        if result.get('error'):
            print(f"   Error: {result['error']}")

    # Display final application state
    final_session = await session_service.get_session(
        app_name="food_delivery_app",
        user_id="test_user_enhanced",
        session_id="enhanced_workflow_test"
    )

    if final_session:
        print(f"\n{'='*30} FINAL APPLICATION STATE {'='*30}")
        state_summary = {
            "Location": final_session.state.get('current_location', {}).get('address', 'Not Set'),
            "Selected Restaurant": final_session.state.get('selected_restaurant', {}).get('name', 'Not Set'),
            "Order Items": len(final_session.state.get('current_order', [])),
            "Order Total": f"${final_session.state.get('order_total', 0):.2f}",
            "Payment Status": final_session.state.get('order_status', 'Not Processed'),
            "Order ID": final_session.state.get('payment_confirmation', {}).get('order_id', 'Not Generated')
        }

        for key, value in state_summary.items():
            print(f"{key}: {value}")

    return workflow_results

# Execute the enhanced rate-limited test
if __name__ == "__main__":
    import asyncio
    enhanced_results = asyncio.run(test_complete_food_delivery_workflow_enhanced_rate_limiting())


STARTING ENHANCED RATE-LIMITED FOOD DELIVERY APPLICATION TEST
⏰ Waiting 15 seconds before starting to respect rate limits...

==================== STEP 1: LOCATION DETECTION ====================
Query: I'm hungry and want to order food from downtown


✅ Delegated to: location_agent
--- Tool: get_user_location called with input: downtown ---
✅ Delegated to: location_agent
✅ Delegated to: location_agent
⏱️ Step completed in 2.4 seconds
Status: PASS
Response: OK. I have your location as 123 Downtown Ave, New York, NY.


==================== STEP 2: RESTAURANT DISCOVERY ====================
Query: Find Italian restaurants with good ratings near me
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: restaurant_discovery_agent
--- Tool: get_restaurants called for location: 123 Downtown Ave, New York, NY ---
✅ Delegated to: restaurant_discovery_agent
✅ Delegated to: restaurant_discovery_agent
⏱️ Step completed in 2.4 seconds
Status: PASS
Response: OK. I found one Italian restaurant near you: Mario's Pizza, with a rating of 4.5.


==================== STEP 3: RESTAURANT SELECTION ====================
Query: I want to order from Mario's Pizza
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: restaurant_selection_agent
--- Tool: select_restaurant called with restaurant: Mario's Pizza ---
✅ Delegated to: restaurant_selection_agent
✅ Delegated to: restaurant_selection_agent
⏱️ Step completed in 2.9 seconds
Status: PASS
Response: OK. You have selected Mario's Pizza.


==================== STEP 4: MENU RETRIEVAL ====================
Query: Show me the menu for Mario's Pizza
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: menu_retrieval_agent
--- Tool: get_menu called for Mario's Pizza ---
✅ Delegated to: menu_retrieval_agent
✅ Delegated to: menu_retrieval_agent
⏱️ Step completed in 2.5 seconds
Status: PASS
Response: OK. Here is the menu for Mario's Pizza:

Appetizers:
Garlic Bread: Fresh baked with herbs - $8.99
Ca...

==================== STEP 5: MENU SECTION NAVIGATION ====================
Query: I want to see the main dishes section
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: menu_navigation_agent
--- Tool: select_menu_section called with section: mains ---
✅ Delegated to: menu_navigation_agent
✅ Delegated to: menu_navigation_agent
⏱️ Step completed in 2.4 seconds
Status: PASS
Response: OK. Here are the main dishes: Margherita Pizza: Fresh mozzarella, basil - $18.99, Pepperoni Pizza: C...

==================== STEP 6: DISH SELECTION ====================
Query: Add 2 Margherita pizzas to my order with extra cheese
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: dish_selection_agent
--- Tool: select_dishes called for Margherita Pizza, quantity: 2 ---
✅ Delegated to: dish_selection_agent
✅ Delegated to: dish_selection_agent
⏱️ Step completed in 2.4 seconds
Status: PASS
Response: OK. I have added 2 Margherita Pizzas with extra cheese to your order. The total for that is $37.98.


==================== STEP 7: PAYMENT PROCESSING ====================
Query: Process payment with credit card and billing address 123 Main St
⏰ Waiting 10 seconds to respect rate limits...


✅ Delegated to: payment_agent
--- Tool: pay called with payment method: credit_card ---
✅ Delegated to: payment_agent
✅ Delegated to: payment_agent
⏱️ Step completed in 2.6 seconds
Status: PASS
Response: OK. I have processed your payment with credit card and your billing address is 123 Main St. Your ord...

ENHANCED RATE-LIMITED TEST RESULTS SUMMARY
Overall Success Rate: 7/7 (100.0%)
✅ Step 1: Location Detection - PASS
✅ Step 2: Restaurant Discovery - PASS
✅ Step 3: Restaurant Selection - PASS
✅ Step 4: Menu Retrieval - PASS
✅ Step 5: Menu Section Navigation - PASS
✅ Step 6: Dish Selection - PASS
✅ Step 7: Payment Processing - PASS

============================== FINAL APPLICATION STATE ==============================
Location: 123 Downtown Ave, New York, NY
Selected Restaurant: Mario's Pizza
Order Items: 1
Order Total: $37.98
Payment Status: confirmed
Order ID: FD19996
